<a href="https://github.com/tianjianjiang/nlp_data_aug/blob/%232-double_redaction/DataAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare

In [0]:
# Ensure GPU spec; T4 is for colab and one can change it for another env.
gpu_list = !nvidia-smi -L
if gpu_list[0].startswith('NVIDIA-SMI has failed'):
  print('Runtime type should be GPU.')
elif not gpu_list[0].startswith('GPU 0: Tesla T4'):
  display(gpu_list)
  print('Please reset all runtimes. We need a Tesla T4 to reproduce the experiments!')
else:
  display(gpu_list)

In [0]:
%%capture pip_logs
!pip install -U fastai==1.0.55 ipyexperiments jupyter-console==5.2.0 coveralls coverage datascience albumentations

In [0]:
import gc
import math
from pathlib import Path
import random

import numpy as np
import torch
from google.colab import drive

from fastai import basic_data, basic_train, core
from fastai import *
from fastai.callbacks import CSVLogger
from fastai.core import plt
from fastai.text import *
from fastprogress import fastprogress

from ipyexperiments import *

In [0]:
# Not set earlier because pip may require a restart.
SESSN_START_T, = !date +%Y%m%dT%H%M

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
# A special treatment for colab to decrease network traffic.
fastprogress.NO_BAR = True
master_bar, progress_bar = fastprogress.force_console_behavior()
basic_train.master_bar, basic_train.progress_bar = master_bar, progress_bar
basic_data.master_bar, basic_data.progress_bar = master_bar, progress_bar
dataclass.master_bar, dataclass.progress_bar = master_bar, progress_bar
text.master_bar, text.progress_bar = master_bar, progress_bar
text.data.master_bar, text.data.progress_bar = master_bar, progress_bar
core.master_bar, core.progress_bar = master_bar, progress_bar

In [0]:
COLAB_CONTENT_DIR_P = Path('/content')
GD_DIR_P = COLAB_CONTENT_DIR_P / 'gdrive'
drive.mount(str(GD_DIR_P), force_remount=True)

In [0]:
BASE_DIR_P = GD_DIR_P / 'My Drive/imdb'
BASE_DIR_P.mkdir(parents=True, exist_ok=True)
DATA_DIR_P = BASE_DIR_P / 'data'
DATA_DIR_P.mkdir(parents=True, exist_ok=True)
MDLS_DIR_P = BASE_DIR_P / 'models'
MDLS_DIR_P.mkdir(parents=True, exist_ok=True)
LOGS_DIR_P = BASE_DIR_P / 'logs'
LOGS_DIR_P.mkdir(parents=True, exist_ok=True)

FASTAI_DATA_DIR_P = Path('/root/.fastai/data')
FASTAI_DATA_DIR_P.mkdir(parents=True, exist_ok=True)

COLAB_DATA_DIR_P = COLAB_CONTENT_DIR_P / 'data'
if not COLAB_DATA_DIR_P.is_symlink():
  COLAB_DATA_DIR_P.symlink_to(FASTAI_DATA_DIR_P)
if (COLAB_CONTENT_DIR_P / 'sample_data').exists():
  !set -x; rm -rf /content/sample_data/

# Download Data Once

In [0]:
path = untar_data(URLs.IMDB,dest=FASTAI_DATA_DIR_P)

In [0]:
path.ls()

[PosixPath('data/imdb/test'),
 PosixPath('data/imdb/README'),
 PosixPath('data/imdb/unsup'),
 PosixPath('data/imdb/imdb.vocab'),
 PosixPath('data/imdb/tmp_lm'),
 PosixPath('data/imdb/models'),
 PosixPath('data/imdb/train'),
 PosixPath('data/imdb/imdb'),
 PosixPath('data/imdb/tmp_clas')]

# Asir's Experiments

In [0]:
data_lm = load_data(path,'imdb')

In [0]:
data_lm.show_batch()

idx,text
0,"new . xxmaj have we all forgotten xxmaj de xxmaj sica 's "" xxmaj bicycle xxmaj thief "" ? xxmaj or anything by xxmaj hitchcock ? \n \n xxmaj so all we get is the extended metaphor of the juvenile as xxmaj truffaut , who spends all his free time ' screwing up his eyes ' at the movies , who wrecks schoolroom discipline , gets accused of xxunk"
1,"film and plays his usual , smarmy , egotistical , snotty character that was actually endearing in "" xxmaj pretty xxmaj in xxmaj pink "" and has xxup not been amusing ever since . xxmaj grating does n't even begin to describe his performance . xxmaj ricky ( xxmaj mark xxmaj xxunk ) is a muscular , blonde , struggling actor who ( gasp ! ) is only worried about"
2,"xxmaj clan , they 're nasty & they do bad things but they ai n't go no soul . i see a lot of reviews from people that liked this , and i guess i do n't know what i missed , but i found it to be very mediocre & i would n't recommend it to anyone , really . 4 out of 10 . xxbos xxmaj this movie"
3,"see if you can count the cliches . xxmaj and yes , xxmaj blair gets possessed , as if you did n't see xxup that coming down xxmaj main xxmaj street followed by a marching band . \n \n xxmaj no stars . "" xxmaj witchery "" - these witches will give you xxunk . xxbos xxmaj charlotte 's deadly beauty and lethal kicks make the movie cooler than"
4,"stallone , who co - wrote the script with xxmaj james xxmaj cameron ( a long way from the exciting xxup terminator made the previous xxunk seems to have given the xxmaj rambo character as little to say in understandable xxmaj english , and merely comes out with moronic grunts , almost as though he has invented his own brand of patois only understandable to himself . xxmaj maybe his"


In [0]:
import spacy
import random
import nltk 
from nltk.corpus import wordnet 
import en_core_web_lg
import re

In [0]:
def synalter_Noun_Verb(cand_word,POS):
    thresh = -1
    scores = []
    synonyms = wordnet.synsets(cand_word)
    for w2 in synonyms:
        try:
            w1 = wordnet.synset(cand_word+'.'+POS+'.01') 
        #w2 = wordnet.synset(word+'.'+POS+'.01') # n denotes noun 
            if (w1.wup_similarity(w2)>thresh):
                scores.append(w1.wup_similarity(w2))
            return synonyms[np.argmax(scores)].name().split('.')[0].replace('_',' ')
        except:
            if not scores:
                for synset in synonyms:
                    word=synset.name().split('.')[0].replace('_',' ')
                    #print(word)
                    token = nlp(word)
                    token_main = nlp(cand_word)
                    if token.vector_norm and token_main.vector_norm and float(token.similarity(token_main)) > thresh:
                        scores.append(float(token.similarity(token_main)))
            return synonyms[np.argmax(scores)].name().split('.')[0].replace('_',' ')

In [0]:
nlp = spacy.load('en_core_web_lg')

In [0]:
doc = nlp('bright car running')
[tok.pos_ for tok in doc]

['ADJ', 'NOUN', 'VERB']

In [0]:
def change_words(word_str,pos,output_text):
    change_word=synalter_Noun_Verb(word_str,pos)
    if change_word:
        search_word = re.search(r'\b('+word_str+r')\b', output_text)
        Loc = search_word.start()
        output_text = output_text[:int(Loc)] + change_word + output_text[int(Loc) + len(word_str):] 
        
    return output_text

In [0]:
from glob import glob

In [0]:
files = glob(str(path)+'/**/*.txt',recursive=True)
len(files)

107858

In [0]:
files = [f for f in files if '_aug.txt' not in f] #skip already augmented files
len(files)

100002

In [0]:
class text_augmenter(object):
#    def __init__(self):
    
    def __call__(self, fn,i): 
        #print(f'augmenting file {fn}')
        output_text = open(fn,'r').read()
        #print("Sentence: "+text)
        counts = Counter(output_text.split())
        noun = []
        verb = []
        doc = nlp(' '.join(w for w in [key for key,value in counts.items() if value ==1]))
        noun = [tok.text for tok in doc if tok.pos_ == 'NOUN']
        verb = [tok.text for tok in doc if tok.pos_ == 'VERB']
        adj = [tok.text for tok in doc if tok.pos_ == 'ADJ']
        cand_words = noun+verb+adj
        len_all = len(verb+noun+adj)
        random.seed(4)
        temp = random.sample(range(len_all), random.randint(0,int(len_all*0.7)))
        #print([cand_words[i] for i in temp])
        for i in temp:
            word_str = cand_words[i]
            if i<len(verb):output_text = change_words(word_str,'v',output_text)
            elif i<len(noun): output_text = change_words(word_str,'n',output_text)
            else:output_text=change_words(word_str,'a',output_text)
        new_fn = fn.replace('.txt','_aug.txt')
        with open(new_fn,'w') as f:
            f.write(output_text)
        #print(f'saving augmented file to {new_fn}')

In [0]:
parallel(text_augmenter(),files)

In [0]:
files = glob(str(path)+'/**/*.txt',recursive=True)
len(files)

190980

Test out the Augmentation on a Classification Task

In [0]:
bs=64

In [0]:
data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos'])
             #label them all with their folders
             .databunch(bs=bs))

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc')

In [0]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.299670,0.192940,0.925820,05:20


In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.224495,0.165303,0.937353,06:32


In [0]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.245834,0.164227,0.938295,08:42


In [0]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.219863,0.162315,0.940054,11:20
1,0.198715,0.159760,0.940995,10:19


In [0]:
learn.save('clas-aug')

# Mike's Experiments

## Define

In [0]:
IMDB_DATA_IN_COLAB_DIR_P = COLAB_DATA_DIR_P / 'imdb'

In [0]:
BPTT = 80
LM_BS = 128
LM_LR = 0.0251
FW_LM_DBNCH_FILE_S = f'fw_lm_dbnch-b{LM_BS}.pkl'
FW_ENC_NAME = f'fw_enc-b{LM_BS}-lr{LM_LR}'

In [0]:
cf_bs = 64
moms = (0.8, 0.7)
cf_drop_mult = 0.5
cf_wd = 0.1
n_dbnch_wrkrs = 0

FW_CF_DBNCH_FILE_S = f'fw_cf_dbnch-b{cf_bs}.pkl'

In [0]:
# Set a constant seed for every random number generator.
SEED = 42


def reset_all_nondeterministic_states(seed=SEED):
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
  torch.backends.cudnn.deterministic = True  # About 15% slower but...
  torch.backends.cudnn.benchmark = False


def build_cf_databunch(data_dir_p, n_workers, bs, vocab):
  reset_all_nondeterministic_states()
  return (TextList.from_folder(data_dir_p, vocab=vocab)
          .split_by_folder(valid='test')
          .label_from_folder(classes=['neg', 'pos'])
          .databunch(bs=bs, num_workers=n_workers))


def init_cf_learner_with_encoder(dbnch, drop_mult, enc_name, base_path=BASE_DIR_P):
  reset_all_nondeterministic_states()
  cf_learn = text_classifier_learner(dbnch, AWD_LSTM, drop_mult=drop_mult, path=base_path, pretrained=False)
  cf_learn.load_encoder(enc_name)
  return cf_learn


def init_cf_cycles(learner, lr, moms, wd, clbks, n_cycles=1):
  print(f'init cf lr: {lr}')
  reset_all_nondeterministic_states()
  learner.fit_one_cycle(n_cycles, lr, moms=moms, wd=wd, callbacks=clbks)
  return learner


def tune_cf_cycles(
    learner,
    lr,
    moms,
    wd,
    clbks_tuple,
    n_cycles_tuple=(1,1,2),
    freeze_steps=(-2,-3,None),
    lr_decays=(2,2,5)
):
  reset_all_nondeterministic_states()
  for n_cycles, freeze_step, lr_decay, clbks in zip(
      n_cycles_tuple, freeze_steps, lr_decays, clbks_tuple):
    if freeze_step is not None:
      learner.freeze_to(freeze_step)
    else:
      learner.unfreeze()
    lr /= lr_decay
    print(f'tune cf lr: {lr}')
    learner.fit_one_cycle(n_cycles, slice(lr/(2.6**4),lr), moms=moms, wd=wd, callbacks=clbks)
  return learner

In [0]:
def double_redact(txt_p, txt_id, chance, noise='xxunk', seed=SEED):
    random.seed(seed)
#     pick_some = False
#     picks = 1 if pick_some else None
    with open(txt_p, 'r') as txt_f:
        txt = txt_f.read()
        chosen_idx_set_list = []
        tkns = txt.split()
        tkn_cnt = len(tkns)
        tkn_idx_set = set(range(tkn_cnt))
        sample_cnt = math.ceil(tkn_cnt * chance)
#         picked = 0
        while sample_cnt <= tkn_cnt:
            chosen_idx_set = set(random.sample(sorted(tkn_idx_set), k=sample_cnt))
            chosen_idx_set_list += [chosen_idx_set]
            tkn_idx_set -= chosen_idx_set
            tkn_cnt = len(tkn_idx_set)
#             picked += 1
#             if pick_some and picked == picks:
#                 break
        for variation, idx_set in enumerate(chosen_idx_set_list):
            redacted_tkns = tkns.copy()
            redacted_txt_p = txt_p.parent / f'{txt_p.stem}_ag-c{chance}-v{variation}-dr{txt_p.suffix}'
            with open(redacted_txt_p, 'w') as redacted_txt_f:
                for chosen_idx in idx_set:
                    redacted_tkns[chosen_idx] = noise
                redacted_txt_f.write(' '.join(redacted_tkns))

## Augment

In [23]:
orig_trn_txts = list(IMDB_DATA_IN_COLAB_DIR_P.glob(f'./train/*/*[0-9].txt'))
sorted(orig_trn_txts)[:3]

[PosixPath('/content/data/imdb/train/neg/0_3.txt'),
 PosixPath('/content/data/imdb/train/neg/10000_4.txt'),
 PosixPath('/content/data/imdb/train/neg/10001_4.txt')]

In [25]:
# Single-threaded although the func is compatible with fastai's `parallel()`

redact_chance = 0.3
for i, orig_trn_txt in enumerate(orig_trn_txts):
  double_redact(orig_trn_txt, i, redact_chance)
dr_trn_txts = list(IMDB_DATA_IN_COLAB_DIR_P.glob(f'./train/*/*[!0-9].txt'))
sorted(dr_trn_txts)[:9]

[PosixPath('/content/data/imdb/train/neg/0_3_ag-c0.3-v0-dr.txt'),
 PosixPath('/content/data/imdb/train/neg/0_3_ag-c0.3-v1-dr.txt'),
 PosixPath('/content/data/imdb/train/neg/0_3_ag-c0.3-v2-dr.txt'),
 PosixPath('/content/data/imdb/train/neg/10000_4_ag-c0.3-v0-dr.txt'),
 PosixPath('/content/data/imdb/train/neg/10000_4_ag-c0.3-v1-dr.txt'),
 PosixPath('/content/data/imdb/train/neg/10000_4_ag-c0.3-v2-dr.txt'),
 PosixPath('/content/data/imdb/train/neg/10001_4_ag-c0.3-v0-dr.txt'),
 PosixPath('/content/data/imdb/train/neg/10001_4_ag-c0.3-v1-dr.txt'),
 PosixPath('/content/data/imdb/train/neg/10001_4_ag-c0.3-v2-dr.txt')]

In [37]:
n_orig_trn = len(orig_trn_txts)
n_dr_trn = len(dr_trn_txts)
n_all_trn = len(list(IMDB_DATA_IN_COLAB_DIR_P.glob(f'./train/*/*.txt')))
print(f'{n_orig_trn} original\n{n_dr_trn} double-redacted\n{n_all_trn} total')

25000 original
74996 double-redacted
99996 total


In [0]:
reset_all_nondeterministic_states()
fw_lm_dbnch = load_data(DATA_DIR_P, FW_LM_DBNCH_FILE_S, bs=LM_BS, bptt=BPTT, num_workers=n_dbnch_wrkrs)

In [0]:
fw_cf_dbnch_dr = build_cf_databunch(IMDB_DATA_IN_COLAB_DIR_P, n_dbnch_wrkrs, cf_bs, fw_lm_dbnch.vocab)
fw_cf_dbnch_dr_fname = f'fw_cf_dbnch-dr{redact_chance}-b{cf_bs}.pkl'
fw_cf_dbnch_dr.save(DATA_DIR_P / fw_cf_dbnch_dr_fname)

## Fit Double-redacted Data in All Cycles

In [0]:
fw_cf_dr_learn = init_cf_learner_with_encoder(fw_cf_dbnch_dr, cf_drop_mult, FW_ENC_NAME)

### Init-fit

In [49]:
cf_lr = 5.18e-2
init_fw_cf_dr_log_p = LOGS_DIR_P / f'{SESSN_START_T}_history-init_fw_cf-dr{redact_chance}-b{cf_bs}-lr{cf_lr}'
init_fw_cf_dr_clbks = [CSVLogger(fw_cf_dr_learn, init_fw_cf_dr_log_p, append=True)]
fw_cf_dr_learn = init_cf_cycles(fw_cf_dr_learn, cf_lr, moms, cf_wd, init_fw_cf_dr_clbks)

init cf lr: 0.0518
epoch     train_loss  valid_loss  accuracy  time    
0         0.278960    0.173263    0.933920  11:36     



#### Comparing with baselines of init. cycle
----
    Mike
    0         0.222152    0.176532    0.934160  04:57

----
    fastai example
    0         0.246949    0.180387    0.931840  01:14

### Fine-tune

In [50]:
tune_fw_cf_dr_clbks_tuple = (
    [CSVLogger(
        fw_cf_dr_learn,
        LOGS_DIR_P / f'{SESSN_START_T}_history-tune_fw_cf-dr{redact_chance}-b{cf_bs}-p{period}',
        append=True)]
    for period in range(1,4)
)
fw_cf_dr_learn = tune_cf_cycles(fw_cf_dr_learn, cf_lr, moms, cf_wd, tune_fw_cf_dr_clbks_tuple)
# fw_cf_dr_learn.save(fw_cf_dr_fname)
# (fw_cf_dr_learn.path/fw_cf_learn.model_dir).ls()

tune cf lr: 0.0259
epoch     train_loss  valid_loss  accuracy  time    
0         0.206510    0.137578    0.949840  13:55     
tune cf lr: 0.01295
epoch     train_loss  valid_loss  accuracy  time    
0         0.170256    0.135338    0.950080  18:30     
tune cf lr: 0.00259
epoch     train_loss  valid_loss  accuracy  time    
0         0.118340    0.146070    0.949160  28:36     
1         0.061189    0.177552    0.949440  26:11     


#### Comparing with baselines of fine-tuning cycles
----
    Mike
    0         0.190170    0.153062    0.944160  05:36
    0         0.180019    0.138620    0.949240  08:08
    0         0.130078    0.143215    0.945520  09:48
    1         0.083496    0.142682    0.949720  09:03

----
    fastai example
    0         0.206164    0.152391    0.945360  01:28
    0         0.181309    0.141463    0.948080  02:30
    0         0.123944    0.145212    0.948840  03:18
    1         0.072845    0.155692    0.949560  03:00


In [0]:
fw_cf_dr_learn.export(MDLS_DIR_P / f'export-fw_cf-dr{redact_chance}-b{cf_bs}-lr{cf_lr}')

In [0]:
fw_cf_dr_learn.destroy(); del fw_cf_dr_learn; gc.collect()

## Fit Double-redacted Data in Middle Cycles (3-Act Structure)

### Act-1: Setup on Orig. Data

In [0]:
reset_all_nondeterministic_states()
fw_cf_dbnch = load_data(DATA_DIR_P, FW_CF_DBNCH_FILE_S, bs=cf_bs, num_workers=n_dbnch_wrkrs)

In [56]:
# Trying to reproduce the result here; next time just loading the model will do.
a1_fw_cf_learn = init_cf_learner_with_encoder(fw_cf_dbnch, cf_drop_mult, FW_ENC_NAME)
a1_fw_cf_log_p = LOGS_DIR_P / f'{SESSN_START_T}_history-act1_fw_cf-b{cf_bs}-lr{cf_lr}'
a1_fw_cf_clbks = [CSVLogger(a1_fw_cf_learn, a1_fw_cf_log_p, append=True)]
a1_fw_cf_learn = init_cf_cycles(a1_fw_cf_learn, cf_lr, moms, cf_wd, a1_fw_cf_clbks)

init cf lr: 0.0518
epoch     train_loss  valid_loss  accuracy  time    
0         0.222152    0.176532    0.934160  04:20     


In [0]:
FW_CF_SETUP_NAME = f'fw_cf-act1-b{cf_bs}-lr{cf_lr}'
a1_fw_cf_learn.save(FW_CF_SETUP_NAME)

In [0]:
a1_fw_cf_learn.destroy(); del a1_fw_cf_learn; gc.collect()

### Act-2: Confrontation on Double-redacted Data

In [0]:
a2_fw_cf_dr_learn = init_cf_learner_with_encoder(fw_cf_dbnch_dr, cf_drop_mult, FW_ENC_NAME)

In [0]:
a2_fw_cf_dr_learn = a2_fw_cf_dr_learn.load(FW_CF_SETUP_NAME)

In [61]:
a2_tune_fw_cf_dr_clbks_tuple = (
    [CSVLogger(
        a2_fw_cf_dr_learn,
        LOGS_DIR_P / f'{SESSN_START_T}_history-tune_fw_cf-act2-dr{redact_chance}-b{cf_bs}-p{period}',
        append=True)]
    for period in range(1,3)
)
a2_fw_cf_dr_learn = tune_cf_cycles(
    a2_fw_cf_dr_learn,
    cf_lr,
    moms,
    cf_wd,
    a2_tune_fw_cf_dr_clbks_tuple,
    n_cycles_tuple=(1,1),
    freeze_steps=(-2,-3),
    lr_decays=(2,2)
)

tune cf lr: 0.0259
epoch     train_loss  valid_loss  accuracy  time    
0         0.210150    0.137580    0.949360  13:56     
tune cf lr: 0.01295
epoch     train_loss  valid_loss  accuracy  time    
0         0.170021    0.137166    0.949240  18:29     


In [0]:
FW_CF_CONFRONTATION_NAME = f'fw_cf-act2-dr{redact_chance}-b{cf_bs}-lr{cf_lr}'
a2_fw_cf_dr_learn.save(FW_CF_CONFRONTATION_NAME)

In [0]:
a2_fw_cf_dr_learn.destroy(); del a2_fw_cf_dr_learn; gc.collect()

### Act-3: Resolution on Orig. Data

In [0]:
a3_fw_cf_learn = init_cf_learner_with_encoder(fw_cf_dbnch, cf_drop_mult, FW_ENC_NAME)

In [0]:
a3_fw_cf_learn = a3_fw_cf_learn.load(FW_CF_CONFRONTATION_NAME)

In [0]:
a3_tune_fw_cf_clbks_tuple = (
    [CSVLogger(
        a3_fw_cf_learn,
        LOGS_DIR_P / f'{SESSN_START_T}_history-tune_fw_cf-act3-b{cf_bs}-p3',
        append=True)],
)
a3_fw_cf_learn = tune_cf_cycles(
    a3_fw_cf_learn,
    cf_lr,
    moms,
    cf_wd,
    a3_tune_fw_cf_clbks_tuple,
    n_cycles_tuple=(2,),
    freeze_steps=(None,),
    lr_decays=(2*2*5,)
)

tune cf lr: 0.00259
epoch     train_loss  valid_loss  accuracy  time    


In [0]:
FW_CF_RESOLUTION_NAME = f'fw_cf-act3-b{cf_bs}-lr{cf_lr}'
a3_fw_cf_learn.save(FW_CF_RESOLUTION_NAME)

In [0]:
a3_fw_cf_learn.destroy(); del a3_fw_cf_learn; gc.collect()